In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r'data/weekly_colombia.csv')

In [3]:
df = df[(df['year'] < 2024) | ((df['year'] == 2024) & (df['month'] < 7))]


In [4]:
df.tail()

,country,brand,year,month,week_of_month,meaning,difference,salience,power,premium,...,brand_distribution_growth,brand_dist_growth_index,weighted_dist,relative_discount_depth_vs_category,nr_to_maco_proportion_y,avg_prcp,avg_temp,avg_min_temp,avg_max_temp,avg_temp_diff
6547,COLOMBIA,TECATE,2024,5,4,70.878677,65.365826,85.32297,1.098873,0.818016,...,0.34828,10.399416,0.145054,1.415595,2.506602,1.233889,72.132712,64.223043,80.042380,15.819337
6548,COLOMBIA,TECATE,2024,6,1,70.878677,65.365826,85.32297,1.098873,0.818016,...,0.34828,10.399416,0.145054,1.415595,2.506602,0.770347,71.743400,64.515887,78.970914,14.455027
6549,COLOMBIA,TECATE,2024,6,2,70.878677,65.365826,85.32297,1.098873,0.818016,...,0.34828,10.399416,0.145054,1.415595,2.506602,0.411286,71.519258,64.217968,78.820548,14.602580
6550,COLOMBIA,TECATE,2024,6,3,70.878677,65.365826,85.32297,1.098873,0.818016,...,0.34828,10.399416,0.145054,1.415595,2.506602,0.691735,71.169496,63.766755,78.572237,14.805481
6551,COLOMBIA,TECATE,2024,6,4,70.878677,65.365826,85.32297,1.098873,0.818016,...,0.34828,10.399416,0.145054,1.415595,2.506602,0.632111,70.090042,62.945359,77.234726,14.289367


In [5]:
# Group by brand, year, and quarter, and calculate average power at year-quarter level
def get_brand_quarterly_avg_power(df):
    """Return DataFrame with avg power per brand, year, quarter."""
    df = df.copy()
    # Create 'quarter' column from month
    df['quarter'] = ((df['month'] - 1) // 3 + 1)
    grouped = df.groupby(['brand', 'year', 'quarter'], as_index=False)['power'].mean()
    grouped = grouped.rename(columns={'power': 'avg_power'})
    return grouped

brand_quarterly_avg_power = get_brand_quarterly_avg_power(df)
print(brand_quarterly_avg_power.head())


    brand  year  quarter  avg_power
0  AGUILA  2020        1  13.461986
1  AGUILA  2020        2  13.461986
2  AGUILA  2020        3  10.139863
3  AGUILA  2020        4  10.139863
4  AGUILA  2021        1  13.729226


In [7]:
brand_quarterly_avg_power.tail()

,brand,year,quarter,avg_power
445,TECATE,2023,2,0.679807
446,TECATE,2023,3,0.754853
447,TECATE,2023,4,0.845267
448,TECATE,2024,1,0.870210
449,TECATE,2024,2,1.098873


In [10]:
# Calculate average percentage change in power per brand, overall, and after 2023

# Compute per-brand average percentage change in power (quarterly)
brand_pct_changes = []
for brand, group in brand_quarterly_avg_power.groupby('brand'):
    group_sorted = group.sort_values(['year', 'quarter'])
    avg_power_prev = group_sorted['avg_power'].shift(1)
    pct_changes = (group_sorted['avg_power'] - avg_power_prev) / avg_power_prev * 100
    pct_changes = pct_changes.dropna()
    avg_pct_change = pct_changes.mean()
    brand_pct_changes.append((brand, avg_pct_change))
    print(f"Brand: {brand:15s} | Avg % change in power: {avg_pct_change:.2f}%")

# Overall average percentage change in power (all brands, all quarters)
all_sorted = brand_quarterly_avg_power.sort_values(['brand', 'year', 'quarter'])
all_avg_power_prev = all_sorted['avg_power'].shift(1)
all_pct_changes = (all_sorted['avg_power'] - all_avg_power_prev) / all_avg_power_prev * 100
# Only consider changes within the same brand
all_pct_changes = all_pct_changes[all_sorted['brand'] == all_sorted['brand'].shift(1)]
overall_avg_pct_change = all_pct_changes.dropna().mean()
print(f"\nOverall avg % change in power: {overall_avg_pct_change:.2f}%")

# Average percentage change in power after 2023 (i.e., for year >= 2023)
after_2023 = brand_quarterly_avg_power[brand_quarterly_avg_power['year'] >= 2023]
after_2023_pct_changes = []
for brand, group in after_2023.groupby('brand'):
    group_sorted = group.sort_values(['year', 'quarter'])
    avg_power_prev = group_sorted['avg_power'].shift(1)
    pct_changes = (group_sorted['avg_power'] - avg_power_prev) / avg_power_prev * 100
    pct_changes = pct_changes.dropna()
    after_2023_pct_changes.extend(pct_changes.tolist())
if after_2023_pct_changes:
    avg_pct_change_after_2023 = sum(after_2023_pct_changes) / len(after_2023_pct_changes)
    print(f"Avg % change in power after 2023: {avg_pct_change_after_2023:.2f}%")
else:
    print("No data for quarters after 2023 to compute avg % change in power.")


Brand: AGUILA          | Avg % change in power: 1.21%
Brand: AGUILA LIGHT    | Avg % change in power: -2.48%
Brand: AMSTEL          | Avg % change in power: 26.92%
Brand: ANDINA          | Avg % change in power: 2.12%
Brand: BBC (BOGOTÁ BEER COMPANY) | Avg % change in power: 5.70%
Brand: BECK'S PILS/ BECK'S | Avg % change in power: 22.84%
Brand: BUSCH LIGHT     | Avg % change in power: 10.70%
Brand: COLA Y POLA     | Avg % change in power: 11.69%
Brand: COORS LIGHT     | Avg % change in power: 4.94%
Brand: COSTENA         | Avg % change in power: 4.61%
Brand: COSTENA BACANA  | Avg % change in power: 7.10%
Brand: COSTENITA       | Avg % change in power: 10.03%
Brand: FAMILIA BUDWEISER | Avg % change in power: 7.91%
Brand: FAMILIA CLUB COLOMBIA | Avg % change in power: -0.94%
Brand: FAMILIA CORONA  | Avg % change in power: 3.09%
Brand: FAMILIA PILSEN  | Avg % change in power: -1.82%
Brand: FAMILIA POKER   | Avg % change in power: -0.81%
Brand: FAMILIA STELLA ARTOIS | Avg % change in powe